In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
main_url = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'

In [246]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [4]:
driver.get(main_url)

In [8]:
import math
def load_all_pages(driver, partido):
    time.sleep(5)
    if any(re.findall('No hay intervenciones',driver.find_element_by_id('navega').text)):
        print('no intervenciones')
    else:
        try:            
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            while not any(number_of_intervenciones):
                time.sleep(1)
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            else:
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))

            number_of_loops = math.ceil(int(number_of_intervenciones)/20)

            if number_of_loops == 1:
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))
                else:
                    return(obtain_metadata(driver, partido))
            else:
                all_data = []
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))

                else:
                    all_data.append(obtain_metadata(driver, partido))
                    r = 20
                    for i in range(1,number_of_loops):
                        time.sleep(3)
                        driver.find_element_by_id('next').click()
                        elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                        e = int(elements_to_scrape) - r
                        entries_found = len(driver.find_elements_by_class_name('NoI'))
                        while entries_found < e:
                            time.sleep(1)
                            entries_found = len(driver.find_elements_by_class_name('NoI'))
                        else:
                            all_data.append(obtain_metadata(driver, partido))
                        r += 20
                    return(pd.concat(all_data))
        except:
            time.sleep(5)
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            while not any(number_of_intervenciones):
                time.sleep(1)
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            else:
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))

            number_of_loops = math.ceil(int(number_of_intervenciones)/20)

            if number_of_loops == 1:
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))
                else:
                    return(obtain_metadata(driver, partido))
            else:
                all_data = []
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))

                else:
                    all_data.append(obtain_metadata(driver, partido))
                    r = 20
                    for i in range(1,number_of_loops):
                        time.sleep(3)
                        driver.find_element_by_id('next').click()
                        elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                        e = int(elements_to_scrape) - r
                        entries_found = len(driver.find_elements_by_class_name('NoI'))
                        while entries_found < e:
                            time.sleep(1)
                            entries_found = len(driver.find_elements_by_class_name('NoI'))
                        else:
                            all_data.append(obtain_metadata(driver, partido))
                        r += 20
                    return(pd.concat(all_data))

In [7]:
def obtain_metadata(driver, party):
    all_data = []
    
    years = []
    tipo_sesion = []
    periodo = []
    fecha = []
    diario = []
    tema = []
    link_speech = []
    
    for i in driver.find_element_by_id('output').find_elements_by_class_name('YearD'):
        years.append(i.text)
    
    for i in driver.find_element_by_id('output').find_elements_by_class_name('FechaD'):
        fecha.append(i.text)
        
    for i in driver.find_element_by_id('output').find_elements_by_class_name('TemaI'):
        tema.append(i.text)
        link_speech.append(i.find_element_by_css_selector('a').get_attribute('href'))
    
    df = pd.DataFrame({'year_session':years, 
                       'fecha': fecha, 
                       'tema':tema,
                       'link_speech':link_speech
                        })
    df['partido'] = party
    df['name'] = driver.find_element_by_id('Escudo').text
    return(df)

In [227]:
def obtain_allData(driver, partido):
    time.sleep(40)
    names_deputies_party = []
    partido_list = []
    for i in driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option'):
        names_deputies_party.append(i.text)
        partido_list.append(partido)
        
    all_d = []
    for i, e in tqdm(zip(driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option'), names_deputies_party), total = len(names_deputies_party)):
        i.click()
        time.sleep(5)
        all_d.append(load_all_pages(driver, partido))
    return(all_d)

## Legislatura 64

In [551]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [552]:
### Loop to all Parties
url_legislatura64 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid3.xql?legis=LXIV'
driver.get(url_legislatura64)

In [553]:
num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

In [562]:
name_parties[4:]

['Partido del Trabajo',
 'Movimiento Ciudadano',
 'Partido de la Revolución Democrática',
 'Partido Verde Ecologista de México',
 'Sin Partido']

In [ ]:
all_dataLegislatura = []

In [563]:
for x, y in tqdm(zip(range(num_parties)[4:], name_parties[4:]), total = len(name_parties[4:])):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura64)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
        
    all_dataLegislatura.append(obtain_allData(driver, y))
    driver.close()

  0%|          | 0/5 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 20%|██        | 1/5 [06:04<24:17, 364.44s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 40%|████      | 2/5 [10:13<16:29, 329.75s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 60%|██████    | 3/5 [14:22<10:10, 305.48s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 80%|████████  | 4/5 [16:21<04:09, 249.64s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 5/5 [16:36<00:00, 199.31s/it]


In [575]:
all_d_faltten = []
for i in all_dataLegislatura:
    all_d_faltten.append(pd.concat(i))

In [576]:
all_d_faltten = pd.concat(all_d_faltten).reset_index().drop(['index'], axis = 1)

In [577]:
len(all_d_faltten)

8332

In [578]:
all_d_faltten.to_csv('./../../data/diputados/floor-speech/data_links_speeches64.csv', encoding = 'utf8')

### Legislatura 63

In [592]:
### Loop to all Parties
url_legislatura63 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXIII#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura63)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura63 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura63)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()

    all_dataLegislatura63.append(obtain_allData(driver, y))
    driver.close()

all_d_faltten63 = []
for i in all_dataLegislatura63:
    all_d_faltten63.append(pd.concat(i))

all_d_faltten63 = pd.concat(all_d_faltten63).reset_index().drop(['index'], axis = 1)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/9 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 11%|█         | 1/9 [56:02<7:28:17, 3362.17s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 22%|██▏       | 2/9 [1:17:06<5:18:49, 2732.77s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 33%|███▎      | 3/9 [1:31:11<3:36:39, 2166.57s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


...

no intervenciones



 67%|██████▋   | 6/9 [2:04:08<58:09, 1163.33s/it]  [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache

 78%|███████▊  | 7/9 [2:09:29<30:21, 910.61s/it] [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 89%|████████▉ | 8/9 [2:13:54<11:56, 716.87s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 9/9 [2:14:24<00:00, 896.02s/it]


In [594]:
all_d_faltten63.to_csv('./../../data/diputados/floor-speech/data_links_speeches63.csv', encoding = 'utf8')

### Legislatura 62

In [16]:
### Loop to all Parties
url_legislatura62 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura62)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura62 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura62)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura62.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura62.append(obtain_allData(driver, y))
    driver.close()

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/7 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 95%|█████████▌| 207/217 [36:45<01:42, 10.20s/it]

no intervenciones



 96%|█████████▌| 208/217 [36:55<01:31, 10.16s/it]

...


 43%|████▎     | 3/7 [1:31:48<2:11:37, 1974.36s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 57%|█████▋    | 4/7 [1:40:04<1:16:33, 1531.13s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 71%|███████▏  | 5/7 [1:49:08<41:09, 1234.83s/it]  [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 86%|████████▌ | 6/7 [1:56:02<16:28, 988.47s/it] [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 7/7 [2:00:29<00:00, 1032.75s/it]


In [17]:
all_d_faltten62 = []
for i in all_dataLegislatura62:
    all_d_faltten62.append(pd.concat(i))

all_d_faltten62 = pd.concat(all_d_faltten62).reset_index().drop(['index'], axis = 1)

In [18]:
all_d_faltten62.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/01-floor_speech/00-scrape_speeches/links_speeches62.csv', encoding = 'utf8')

### Legislatura 61

In [7]:
# Loop to all Parties
url_legislatura61 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXI#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura61)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura61 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura61)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura61.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura61.append(obtain_allData(driver, y))
    driver.close()

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87]
  0%|          | 0/7 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 79%|███████▉  | 191/241 [36:20<09:09, 10.99s/it]

no intervenciones



 87%|████████▋ | 209/241 [39:59<06:18, 11.83s/it]

...

[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache

 43%|████▎     | 3/7 [1:58:42<2:38:09, 2372.25s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache

 57%|█████▋    | 4/7 [2:04:39<1:28:22, 1767.58s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 71%|███████▏  | 5/7 [2:18:36<49:37, 1488.61s/it]  [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 86%|████████▌ | 6/7 [2:21:07<18:07, 1087.26s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 7/7 [2:23:08<00:00, 1226.95s/it]


In [8]:
all_d_faltten61 = []
for i in all_dataLegislatura61:
    all_d_faltten61.append(pd.concat(i))

all_d_faltten61 = pd.concat(all_d_faltten61).reset_index().drop(['index'], axis = 1)

In [11]:
all_d_faltten61.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-floor_speech/00-scrape_speeches/links_speeches61.csv', encoding = 'utf8')

### Legislatura through Diario de Debates

In [11]:
all_dataLegislatura60 = []

In [105]:
all_dataLegislatura60[-1:]

[  year_session     fecha                                              tema  \
 0           II  20070927  ANALISIS I INFORME DE GOBIERNO. POLITICA SOCI...   
 
                                          link_speech                  partido  \
 0  http://cronica.diputados.gob.mx:8080/exist/sii...  Partido Acción Nacional   
 
                          name  
 0  Jiménez Ramos María Esther  ]

In [221]:
all_dataLegislatura60.append(obtain_metadata(driver, y))

In [248]:
all_dataPRD = []

In [251]:
all_dataPRD.append(obtain_allData(driver, name_parties[1]))

  9%|▉         | 12/127 [03:46<36:14, 18.91s/it]


ValueError: max() arg is an empty sequence

In [238]:
len(driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option')[12:13])

1

In [243]:
driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option')[14].click()

In [258]:
all_dataPRD_batch2[0]

[]

In [245]:
all_dataPRD_batch2.append(load_all_pages(driver, name_parties[1]))

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=52588): Max retries exceeded with url: /session/01c38863e28e4ad1f0483f0e3246fc73/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f90f3cfea60>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [237]:
for i in driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option')[13]:
    all_d = []
    i.click()
    time.sleep(30)
    all_d.append(load_all_pages(driver, name_parties[1]))
all_dataPRD_batch2.append(all_d)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

'Partido Verde Ecologista de México'

In [ ]:
all_data = []

In [290]:
for i in tqdm(driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option')):
    i.click()
    try:
        time.sleep(15)
        all_data.append(load_all_pages(driver, name_parties[7]))
    except:
        time.sleep(20)
        all_data.append(load_all_pages(driver, name_parties[7]))

100%|██████████| 5/5 [02:16<00:00, 27.36s/it]


In [289]:
all_data.append(obtain_metadata(driver, name_parties[6]))

In [293]:
name_parties

['Partido Acción Nacional',
 'Partido de la Revolución Democrática',
 'Partido Revolucionario Institucional',
 'Partido Verde Ecologista de México',
 'Convergencia',
 'Partido del Trabajo',
 'Partido Nueva Alianza',
 'Alternativa']

In [273]:
name_parties[3]

'Partido Verde Ecologista de México'

In [295]:
pd.concat(all_data).to_csv('/Users/cblanesg/Desktop/batch_partidos.csv', encoding = 'utf-8 sig')

In [294]:
pd.concat(all_data)

,year_session,fecha,tema,link_speech,partido,name
0,I,20060926,ANALISIS VI INFORME DE GOBIERNO. POLITICA SOC...,http://cronica.diputados.gob.mx:8080/exist/sii...,Convergencia,Abad de Jesús Juan
1,II,20070912,LEY ORGANICA DEL CONGRESO GENERAL DE LOS ESTA...,http://cronica.diputados.gob.mx:8080/exist/sii...,Convergencia,Abad de Jesús Juan
2,II,20070913,LEY DEL IMPUESTO A LOS DEPOSITOS EN EFECTIVO ...,http://cronica.diputados.gob.mx:8080/exist/sii...,Convergencia,Abad de Jesús Juan
3,II,20070927,ANALISIS I INFORME DE GOBIERNO. POLITICA SOCI...,http://cronica.diputados.gob.mx:8080/exist/sii...,Convergencia,Abad de Jesús Juan
4,II,20071206,LEY GENERAL PARA EL CONTROL DEL TABACO - LEY ...,http://cronica.diputados.gob.mx:8080/exist/sii...,Convergencia,Abad de Jesús Juan
...,...,...,...,...,...,...
7,II,20071009,ESTADO DE OAXACA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio
8,II,20071018,CONSTITUYENTES DE APATZINGAN,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio
9,II,20080205,AGENDA LEGISLATIVA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio
10,II,20080221,DIA INTERNACIONAL DE LA LENGUA MATERNA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio


In [222]:
pd.concat(all_dataLegislatura60).to_csv('/Users/cblanesg/Desktop/batch_pan.csv',encoding = 'utf-8 sig')

In [224]:
pd.read_csv('/Users/cblanesg/Desktop/batch_pan.csv')

,Unnamed: 0,year_session,fecha,tema,link_speech,partido,name
0,0,I,20060905,FENOMENOS CLIMATOLOGICOS,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
1,1,I,20061024,ESTADO DE SONORA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
2,2,II,20070913,LEY DEL IMPUESTO EMPRESARIAL A TASA UNICA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
3,3,II,20071211,CODIGO FEDERAL DE INSTITUCIONES Y PROCEDIMIEN...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
4,4,II,20080619,ARTICULO 88 CONSTITUCIONAL,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
...,...,...,...,...,...,...,...
1223,0,I,20061003,ANALISIS VI INFORME DE GOBIERNO. POLITICA INT...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Zambrano Elizondo Javier Martín
1224,1,I,20061026,DEFINICION DEL FUTURO DE MEXICO,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Zambrano Elizondo Javier Martín
1225,2,III,20081007,LEY FEDERAL DE JUEGOS Y SORTEOS - LEY DE AGUA...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Zambrano Elizondo Javier Martín
1226,3,III,20081104,LEY DE CIENCIA Y TECNOLOGIA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Zambrano Elizondo Javier Martín


In [312]:
all_data_parties = []

In [316]:
all_data_parties.append(pd.read_csv('/Users/cblanesg/Desktop/batch_partidos.csv'))

In [304]:
pd.concat(all_data_parties)

,Unnamed: 0,year_session,fecha,tema,link_speech,partido,name,partido.1
0,0,I,20060905,FENOMENOS CLIMATOLOGICOS,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo,NaN
1,1,I,20061024,ESTADO DE SONORA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo,NaN
2,2,II,20070913,LEY DEL IMPUESTO EMPRESARIAL A TASA UNICA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo,NaN
3,3,II,20071211,CODIGO FEDERAL DE INSTITUCIONES Y PROCEDIMIEN...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo,NaN
4,4,II,20080619,ARTICULO 88 CONSTITUCIONAL,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo,NaN
...,...,...,...,...,...,...,...,...
1068,7,II,20071009,ESTADO DE OAXACA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio,NaN
1069,8,II,20071018,CONSTITUYENTES DE APATZINGAN,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio,NaN
1070,9,II,20080205,AGENDA LEGISLATIVA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio,NaN
1071,10,II,20080221,DIA INTERNACIONAL DE LA LENGUA MATERNA,http://cronica.diputados.gob.mx:8080/exist/sii...,Alternativa,Hernández Valadés Delio,NaN


In [311]:
pri = pd.read_csv('/Users/cblanesg/Desktop/batch_pri.csv').drop(['partido'], axis = 1).rename(columns = {'partido.1':'partido'})

In [317]:
all_data_parties.append(pri)

In [323]:
pd.concat(all_data_parties).drop(['Unnamed: 0'], axis = 1).to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-floor_speech/00-raw/links_speeches61.csv')

,year_session,fecha,tema,link_speech,partido,name
0,I,20060905,FENOMENOS CLIMATOLOGICOS,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
1,I,20061024,ESTADO DE SONORA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
2,II,20070913,LEY DEL IMPUESTO EMPRESARIAL A TASA UNICA,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
3,II,20071211,CODIGO FEDERAL DE INSTITUCIONES Y PROCEDIMIEN...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
4,II,20080619,ARTICULO 88 CONSTITUCIONAL,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Acción Nacional,Aranda Orozco Gerardo
...,...,...,...,...,...,...
1092,II,20071212,ARTICULO 123 CONSTITUCIONAL,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Revolucionario Institucional,Zataraín González Carlos Ernesto
1093,II,20080311,LEY DEL SISTEMA NACIONAL DE INFORMACION ESTAD...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Revolucionario Institucional,Zataraín González Carlos Ernesto
1094,II,20080326,LEY ORGANICA DEL CONGRESO GENERAL DE LOS ESTA...,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Revolucionario Institucional,Zataraín González Carlos Ernesto
1095,II,20080430,LEY DE FOMENTO PARA LA LECTURA Y EL LIBRO,http://cronica.diputados.gob.mx:8080/exist/sii...,Partido Revolucionario Institucional,Zataraín González Carlos Ernesto


In [12]:
all_dataLegislatura60.append(obtain_allData(driver, y))

  2%|▏         | 5/206 [01:32<1:02:14, 18.58s/it]


ValueError: max() arg is an empty sequence

In [249]:
# Loop to all Parties
url_legislatura61 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LX#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura61)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [9]:

all_dataLegislatura60 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura61)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        time.sleep(40)
        all_dataLegislatura60.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura60.append(obtain_allData(driver, y))
    driver.close()

  0%|          | 0/8 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



  0%|          | 1/206 [00:25<1:26:36, 25.35s/it]A

  0%|          | 0/8 [01:43<?, ?it/s]


ValueError: max() arg is an empty sequence

In [ ]:
all_d_faltten60 = []
for i in all_dataLegislatura60:
    all_d_faltten60.append(pd.concat(i))

all_d_faltten60 = pd.concat(all_d_faltten60).reset_index().drop(['index'], axis = 1)

In [ ]:
all_d_faltten60.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-floor_speech/00-scrape_speeches/links_speeches60.csv', encoding = 'utf8')